In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re

In [3]:
url="https://www.gatesnotes.com/Books#BooksRead"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(
    url,
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [4]:
regex = r"var html = template\((.*?)\);"
scripts  = soup.find_all("script")
found = []
for script in scripts:
    matches = re.finditer(regex, script.text, re.MULTILINE)
    for matchNum, match in enumerate(matches, start=1):
        found.append(match.group(1))

In [5]:
found = [
    json.loads(match) for match in found if match and not match == 'data'
]

In [6]:
books = []
for match in found:
    books.extend(
        match.get('GoodReadsBookItemsList', [])
    )

In [7]:
df = pd.DataFrame(books)

In [8]:
df.head()

,TGN_image,TGN_imageGUID,TGN_review_article,TGN_review_articleURL,author_name,date_added,deactivate_this_item,goodreads_book_id,image_url,isbn,isbn13,large_image_url,link,publication_day,publication_month,publication_year,read_at,small_image_url,title,title_without_series
0,/-/media/Images/GoodReadsBookCovers/Leadership...,"<image mediaid=""{82EB8042-1597-4AE6-9C74-C2350...",,None,Doris Kearns Goodwin,Thu Jan 03 09:18:25 -0800 2019,,38657386,https://media.gatesnotes.com/-/media/Images/Go...,1476795924,9781476795928,,https://www.goodreads.com/book/show/38657386-l...,18,9,2018,,https://images.gr-assets.com/books/1531683409s...,Leadership: In Turbulent Times,Leadership: In Turbulent Times
1,/-/media/Images/GoodReadsBookCovers/TheMosquit...,"<image mediaid=""{93FDDF65-D1FD-417E-82AF-4E47F...",,None,Timothy C. Winegard,Fri Mar 01 13:23:57 -0800 2019,,42983957,https://media.gatesnotes.com/-/media/Images/Go...,0735235791,9780735235793,,https://www.goodreads.com/book/show/42983957-t...,6,8,2019,,,The Mosquito,The Mosquito: A Human History of Our Deadliest...
2,/-/media/Images/GoodReadsBookCovers/Outbreak-C...,"<image mediaid=""{DE263EBD-0859-4D8E-9120-C183E...",,None,Pardis Sabeti,Mon Jun 03 09:10:46 -0700 2019,,39217055,https://media.gatesnotes.com/-/media/Images/Go...,0674976118,9780674976115,,https://www.goodreads.com/book/show/39217055-o...,26,11,2018,,https://images.gr-assets.com/books/1524513965s...,Outbreak Culture,Outbreak Culture: The Ebola Crisis and the Nex...
3,/-/media/Images/GoodReadsBookCovers/cloud-atla...,"<image mediaid=""{00D1BFE4-51A4-43D3-82EE-22D32...",,None,David Mitchell,Fri Mar 01 13:23:57 -0800 2019,,49628,https://media.gatesnotes.com/-/media/Images/Go...,0375507256,9780375507250,,https://www.goodreads.com/book/show/49628.Clou...,17,8,2004,,,Cloud Atlas,Cloud Atlas
4,/-/media/Images/GoodReadsBookCovers/The-Three-...,"<image mediaid=""{C171D5E0-DA69-44AC-9B5D-9E265...",,None,Cixin Liu,Fri Mar 01 13:23:57 -0800 2019,,20518872,https://media.gatesnotes.com/-/media/Images/Go...,0765377063,9780765377067,,https://www.goodreads.com/book/show/20518872-t...,11,11,2014,,,The Three-Body Problem,The Three-Body Problem


In [9]:
df['read_at'] = pd.to_datetime(df['read_at'])
df['date_added'] = pd.to_datetime(df['date_added'])

In [10]:
columns = ['title', 'title_without_series', 'author_name', 'read_at', 'date_added', 'isbn', 'isbn13', 'goodreads_book_id', 'TGN_review_articleURL']
df = df[columns].sort_values('read_at', ascending=False)
df.head()

,title,title_without_series,author_name,read_at,date_added,isbn,isbn13,goodreads_book_id,TGN_review_articleURL
21,Growth,Growth: From Microorganisms to Megacities,Vaclav Smil,2019-12-04 08:00:00,2019-06-28 19:20:09,0262042835,9780262042833,44512537,/Books/Growth
52,Prepared,Prepared: What Kids Need for a Fulfilled Life,Diane Tavenner,2019-12-04 08:00:00,2019-05-01 17:42:46,1984826069,9781984826060,38212158,/Books/Prepared
135,These Truths,These Truths: A History of the United States,Jill Lepore,2019-12-04 08:00:00,2019-05-01 17:42:46,0393635244,9780393635249,38212158,/Books/These-Truths
58,Dad Is Fat,Dad Is Fat,Jim Gaffigan,2019-06-28 19:19:32,2019-06-03 16:11:03,0385349068,9780385349062,16141924,None
25,Upheaval,Upheaval: Turning Points for Nations in Crisis,Jared Diamond,2019-04-29 07:00:00,2018-11-13 00:43:39,0316409138,9780316409131,41716904,/Books/Upheaval


In [11]:
def get_gates_book(url, base='https://www.gatesnotes.com/'):
    
    if url:
        
        url = base + url

        # Make a GET request to fetch the raw HTML content
        html_content = requests.get(
            url,
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        ).text

        # Parse the html content
        soup = BeautifulSoup(html_content, "lxml")
        book = soup.find('div', class_="TGN_site_Article_body")
        text = book.find_all(text=True)
        
        output = ''
        blacklist = [
            '[document]',
            'noscript',
            'header',
            'html',
            'meta',
            'head', 
            'input',
            'script',
            'style'
        ]

        for t in text:
            if t.parent.name not in blacklist:
                output += '{} '.format(t)

        return output

In [13]:
# df['review_text'] = df['TGN_review_articleURL'].apply(lambda x: get_gates_book(x))

In [15]:
df['is_reviewed'] = df['TGN_review_articleURL'].apply(lambda x: True if x else False)

In [16]:
df.head()

,title,title_without_series,author_name,read_at,date_added,isbn,isbn13,goodreads_book_id,TGN_review_articleURL,is_reviewed
21,Growth,Growth: From Microorganisms to Megacities,Vaclav Smil,2019-12-04 08:00:00,2019-06-28 19:20:09,0262042835,9780262042833,44512537,/Books/Growth,True
52,Prepared,Prepared: What Kids Need for a Fulfilled Life,Diane Tavenner,2019-12-04 08:00:00,2019-05-01 17:42:46,1984826069,9781984826060,38212158,/Books/Prepared,True
135,These Truths,These Truths: A History of the United States,Jill Lepore,2019-12-04 08:00:00,2019-05-01 17:42:46,0393635244,9780393635249,38212158,/Books/These-Truths,True
58,Dad Is Fat,Dad Is Fat,Jim Gaffigan,2019-06-28 19:19:32,2019-06-03 16:11:03,0385349068,9780385349062,16141924,None,False
25,Upheaval,Upheaval: Turning Points for Nations in Crisis,Jared Diamond,2019-04-29 07:00:00,2018-11-13 00:43:39,0316409138,9780316409131,41716904,/Books/Upheaval,True
